# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [31]:
# from pymongo import MongoClient
# client = MongoClient("")

In [32]:
from pymongo import MongoClient
import pandas as pd
import time

In [33]:
client = MongoClient("localhost:27017")

In [34]:
client.list_database_names()

['Ironhack', 'admin', 'config', 'local']

In [35]:
db = client["Ironhack"]

In [36]:
c = db.get_collection("companies")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [37]:
list(c.find({"name": "Babelgum"}, {"name": 1, "_id": 0}))

[{'name': 'Babelgum'}]

In [56]:
query = {"name": "Babelgum"}
projection = {"name": 1, "_id": 0}
    
list(c.find(query, projection))


[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [45]:
query = {"number_of_employees": {"$gt": 5000}}
projection = {"name": 1, "_id": 0}
    
result = list(c.find(query, projection))[:5]
result

[{'name': 'Facebook'},
 {'name': 'eBay'},
 {'name': 'Cisco'},
 {'name': 'Yahoo!'},
 {'name': 'Google'}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [59]:
cond_1 = {"founded_year": {"$gt": 1999}}
cond_2 = {"founded_year": {"$lt": 2006}}


query = {"$and": [cond_1, cond_2]}
projection = {"name": 1, "_id": 0, "founded_year": 1}

list(c.find(query, projection).limit(10))


[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [87]:
cond_1 = {"ipo.valuation_amount": {"$gt": 100000000}}
cond_2 = {"founded_year": {"$lt": 2010}} 

query = {"$and": [cond_1, cond_2]}
projection = {"name": 1, "ipo":1, "_id":0}

list(c.find(query, projection))[:5]

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [117]:
cond_1 = {"number_of_employees": {"$lt": 1000}}
cond_2 = {"founded_year": {"$lt": 2005}} 


query = {"$and": [cond_1, cond_2]}
projection = {"name": 1, "_id":0}

list(c.find(query, projection).sort("number_of_employees", 1).limit(10))

[{'name': 'Fox Interactive Media'},
 {'name': 'Ticketmaster'},
 {'name': 'Eurekster'},
 {'name': 'MindTouch'},
 {'name': 'Skype'},
 {'name': 'Monster'},
 {'name': 'EditGrid'},
 {'name': 'Simpy'},
 {'name': 'Compete'},
 {'name': 'stylediary'}]

### 6. All the companies that don't include the `partners` field.

In [109]:
query = {"partners": {"$exists": False}}
projection = {"name": 1, "_id":0}

list(c.find(query, projection))


[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [122]:
query = {"category_code": {"$type": "null"}}
projection = {"name": 1, "_id":0}

list(c.find(query, projection).limit(10))


[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'Touch Clarity'},
 {'name': 'MMDAYS'},
 {'name': 'Inside Group'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [124]:
cond_1 = {"number_of_employees": {"$gte": 100}}
cond_1 = {"number_of_employees": {"$lt": 1000}}

query = {"$and": [cond_1, cond_2]}
projection = {"name": 1, "_id":0}

list(c.find(query, projection).limit(10))

[{'name': 'AdventNet'},
 {'name': 'Digg'},
 {'name': 'Fox Interactive Media'},
 {'name': 'Plaxo'},
 {'name': 'Technorati'},
 {'name': 'AddThis'},
 {'name': 'Meetup'},
 {'name': 'SmugMug'},
 {'name': 'Skype'},
 {'name': 'Pando Networks'}]

### 9. Order all the companies by their IPO price in a descending order.

In [129]:
query = {"ipo": {"$exists": True}}
projection = {"name": 1, "_id":0, "ipo": 1}

list(c.find(query, projection).sort("ipo", -1))

[{'name': 'GREE',
  'ipo': {'valuation_amount': 108960000000.0,
   'valuation_currency_code': 'JPY',
   'pub_year': 2008,
   'pub_month': 12,
   'pub_day': 17,
   'stock_symbol': '3632'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Groupon',
  'ipo': {'valuation_amount': 12800000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NASDAQ:GRPN'}},
 {'name': 'Tencent',
  

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [147]:
query = {}
projection = {"name": 1, "_id":0}

list(c.find(query, projection).sort("number_of_employees", -1).limit(10))

[{'name': 'Siemens'},
 {'name': 'IBM'},
 {'name': 'Toyota'},
 {'name': 'PayPal'},
 {'name': 'Nippon Telegraph and Telephone Corporation'},
 {'name': 'Samsung Electronics'},
 {'name': 'Accenture'},
 {'name': 'Tata Consultancy Services'},
 {'name': 'Flextronics International'},
 {'name': 'Safeway'}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [154]:
query = {"founded_month": {"$gte": 6}}
projection = {"name": 1, "_id":0, "founded_month":1}


list(c.find(query, projection).sort("name", 1).sort("founded_month", 1).limit(1000))

[{'name': 'Digital Base', 'founded_month': 6},
 {'name': 'XDnet', 'founded_month': 6},
 {'name': 'Ruckus Wireless', 'founded_month': 6},
 {'name': 'Songkick', 'founded_month': 6},
 {'name': 'Publictivity', 'founded_month': 6},
 {'name': 'DoggieInk', 'founded_month': 6},
 {'name': 'SeeInside', 'founded_month': 6},
 {'name': 'Fox Interactive Media', 'founded_month': 6},
 {'name': 'MediResource', 'founded_month': 6},
 {'name': 'NetCitadel', 'founded_month': 6},
 {'name': 'youcalc', 'founded_month': 6},
 {'name': 'Cignis', 'founded_month': 6},
 {'name': 'Model Metrics', 'founded_month': 6},
 {'name': 'C-KDeals Technology', 'founded_month': 6},
 {'name': 'CyberSynchs', 'founded_month': 6},
 {'name': 'SyllabusCentral', 'founded_month': 6},
 {'name': 'ceo site pad', 'founded_month': 6},
 {'name': 'Dojo Learning', 'founded_month': 6},
 {'name': 'Sexinema', 'founded_month': 6},
 {'name': 'MeeMix', 'founded_month': 6},
 {'name': 'Boulevard R', 'founded_month': 6},
 {'name': 'LocoGopher', 'founde

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [235]:
cond_1 = {"acquisition.price_amount": {"$gt": 10000000}}
cond_2 = {"acquisition.acquired_year": {"$lt": 2000}} 

query = {"$and": [cond_1, cond_2]}
projection = {"name": 1, "_id":0, "acquisition.acquired_year": 1}

list(c.find(query, projection).sort("acquisition.price_amount", -1))

[{'name': 'Netscape', 'acquisition': {'acquired_year': 1998}},
 {'name': 'Lotus Development Corporation',
  'acquisition': {'acquired_year': 1995}},
 {'name': 'GeoCities', 'acquisition': {'acquired_year': 1999}},
 {'name': 'Berkeley Networks', 'acquisition': {'acquired_year': 1999}},
 {'name': 'Legent Corporation', 'acquisition': {'acquired_year': 1995}},
 {'name': 'MapQuest', 'acquisition': {'acquired_year': 1999}},
 {'name': 'Lightera', 'acquisition': {'acquired_year': 1999}},
 {'name': 'MovieFone', 'acquisition': {'acquired_year': 1999}},
 {'name': 'zip2', 'acquisition': {'acquired_year': 1999}},
 {'name': 'Alexa', 'acquisition': {'acquired_year': 1999}},
 {'name': 'Encompass', 'acquisition': {'acquired_year': 1999}},
 {'name': 'Maxis', 'acquisition': {'acquired_year': 1997}},
 {'name': 'VideoSurf', 'acquisition': {'acquired_year': 1993}},
 {'name': 'VideoSurf', 'acquisition': {'acquired_year': 1993}},
 {'name': 'Four11', 'acquisition': {'acquired_year': 1997}},
 {'name': 'Online An

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [174]:
query = {"acquisition.acquired_year": {"$gt": 2010}} 
projection = {"name": 1, "_id":0, "acquisition": 1}

list(c.find(query, projection).sort("acquisition.price_amount", -1))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name':

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [177]:
query = {}
projection = {"name": 1, "_id":0, "founded_year": 1}

list(c.find(query, projection).sort("founded_year", -1).limit(10))

[{'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [176]:
query = {"founded_day": {"$lte": 7}}
projection = {"name": 1, "_id":0, "founded_day": 1}

list(c.find(query, projection).sort("acquisition.price_amount", -1).limit(10))

[{'name': 'Netscape', 'founded_day': 4},
 {'name': 'PayPal', 'founded_day': 1},
 {'name': 'Zappos', 'founded_day': 1},
 {'name': 'Alibaba', 'founded_day': 1},
 {'name': 'Postini', 'founded_day': 2},
 {'name': 'Danger', 'founded_day': 1},
 {'name': 'Clearwell Systems', 'founded_day': 6},
 {'name': 'PrimeSense', 'founded_day': 1},
 {'name': 'Amobee', 'founded_day': 1},
 {'name': 'Vitrue', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [190]:
cond_1 = {"category_code": "web"}
cond_2 = {"number_of_employees": {"$gt": 4000}}

query = {"$and": [cond_1, cond_2]}
projection = {"name": 1, "_id":0, "category_code": 1, "number_of_employees" :1}

list(c.find(query, projection).sort("number_of_employees", 1).limit(10))

[{'name': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'category_code': 'web', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'name': 'Groupon', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'category_code': 'web', 'number_of_employees': 13600},
 {'name': 'eBay', 'category_code': 'web', 'number_of_employees': 15000},
 {'name': 'Experian', 'category_code': 'web', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [199]:
cond_1 = {"acquisition.price_amount": {"$gt": 10000000}}
cond_2 = {"acquisition.price_currency_code": "EUR"}

query = {"$and": [cond_1, cond_2]}
projection = {"name": 1, "_id":0,"acquisition.price_currency_code":1 }

list(c.find(query, projection).limit(10))

[{'name': 'ZYB', 'acquisition': {'price_currency_code': 'EUR'}},
 {'name': 'Apertio', 'acquisition': {'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online', 'acquisition': {'price_currency_code': 'EUR'}},
 {'name': 'Webedia', 'acquisition': {'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder', 'acquisition': {'price_currency_code': 'EUR'}},
 {'name': 'Tuenti Technologies',
  'acquisition': {'price_currency_code': 'EUR'}},
 {'name': 'BioMed Central', 'acquisition': {'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [214]:
cond_1 = {"acquisitions.acquired_month": {"$gte": 3}}
cond_2 = {"acquisitions.acquired_month": {"$lte": 6}}

query = {"$and": [cond_1, cond_2]}
projection = {"name": 1, "_id":0, "acquisitions": 1}

list(c.find(query, projection).limit(10))

[{'name': 'Facebook',
  'acquisitions': [{'price_amount': None,
    'price_currency_code': 'USD',
    'term_code': 'cash',
    'source_url': 'http://www.techcrunch.com/2007/07/19/breaking-facebook-has-acquired-parakey/',
    'source_description': '',
    'acquired_year': 2007,
    'acquired_month': 7,
    'acquired_day': 1,
    'company': {'name': 'Parakey', 'permalink': 'parakey'}},
   {'price_amount': 47500000,
    'price_currency_code': 'USD',
    'term_code': 'cash_and_stock',
    'source_url': 'http://www.techcrunch.com/2009/08/10/facebook-acquires-friendfeed',
    'source_description': 'Facebook Acquires FriendFeed',
    'acquired_year': 2009,
    'acquired_month': 8,
    'acquired_day': 10,
    'company': {'name': 'FriendFeed', 'permalink': 'friendfeed'}},
   {'price_amount': None,
    'price_currency_code': 'USD',
    'term_code': None,
    'source_url': 'http://techcrunch.com/2010/02/19/octazen-what-the-heck-did-facebook-just-buy-exactly-and-why/',
    'source_description': ''

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [227]:
cond_1 = {"founded_year": {"$gte": 2000, "$lte": 2010}}
cond_2 = {"acquisitions.acquired_year": {"$gte": 2011}}

query = {"$and": [cond_1, cond_2]}
projection = {"name": 1, "_id":0, "founded_year":1}

list(c.find(query, projection))

[{'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Twitter', 'founded_year': 2006},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'OpenX', 'founded_year': 2008},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003},
 {'name': 'YouTube', 'founded_year': 2005},
 {'name': 'blinkx', 'founded_year': 2004},
 {'name': 'Yelp', 'founded_year': 2004},
 {'name': 'Dailymotion', 'founded_year': 2005},
 {'name': 'RockYou', 'founded_year': 2005},
 {'name': 'Meebo', 'founded_year': 2005},
 {'name': 'Eventbrite', 'founded_year': 2006},
 {'name': 'LinkedIn', 'founded_year': 2003},
 {'name': 'Box', 'founded_year': 2005},
 {'name': 'Carbonite', 'founded_year': 2006},
 {'name': 'Conduit', 'founded_year': 2005},
 {'name': 'GigaOM', 'founded_year': 2006},
 {'name': 'PhotoBox', 'founded_year': 2000},
 {'name': 'Bazaarvoice', 'founded_year': 2005},
 {'name': 'spigit', 'founde

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```